In [ ]:
import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
import pytensor

from climepi.epimod._model_fitting import ParameterizedSuitabilityModel

pytensor.config.cxx = "/usr/bin/clang++"


In [ ]:
parameters = {
    "eggs_per_female_per_day": {
        "curve_type": "briere",
        "priors": {
            "steepness": lambda: pm.Gamma("steepness", alpha=2, beta=100),
            "temperature_min": lambda: pm.Gamma(
                "temperature_min", alpha=10, beta=1 / 2
            ),
            "temperature_max": lambda: pm.Gamma(
                "temperature_max", alpha=10, beta=1 / 4
            ),
            "noise_std": lambda: pm.Uniform("noise_std", lower=0, upper=10),
        },
        "attrs": {"long_name": "Eggs per female per day"},
    },
    "development_rate": {
        "curve_type": "briere",
        "priors": {
            "steepness": lambda: pm.Gamma("steepness", alpha=9, beta=100000),
            "temperature_min": lambda: pm.Gamma("temperature_min", alpha=7, beta=1 / 2),
            "temperature_max": lambda: pm.Gamma(
                "temperature_max", alpha=10, beta=1 / 4
            ),
            "noise_std": lambda: pm.Uniform("noise_std", lower=0, upper=1),
        },
        "attrs": {"long_name": "Development rate", "units": "per day"},
    },
    "egg_to_adult_survival_probability": {
        "curve_type": "quadratic",
        "probability": True,
        "priors": {
            "steepness": lambda: pm.Gamma("steepness", alpha=7, beta=1000),
            "temperature_min": lambda: pm.Gamma("temperature_min", alpha=7, beta=1 / 2),
            "temperature_max": lambda: pm.Gamma(
                "temperature_max", alpha=10, beta=1 / 4
            ),
            "noise_std": lambda: pm.Uniform("noise_std", lower=0, upper=5),
        },
        "attrs": {"long_name": "Egg to adult survival probability"},
    },
    "adult_lifespan": {
        "curve_type": "quadratic",
        "priors": {
            "steepness": lambda: pm.Gamma("steepness", alpha=1, beta=2),
            "temperature_min": lambda: pm.Gamma("temperature_min", alpha=5, beta=1 / 2),
            "temperature_max": lambda: pm.Gamma("temperature_max", alpha=9, beta=1 / 5),
            "noise_std": lambda: pm.Uniform("noise_std", lower=0, upper=50),
        },
        "attrs": {
            "long_name": "Adult lifespan",
            "units": "days",
        },
    },
}
data = pd.read_csv("data.csv")

In [ ]:
suitability_model = ParameterizedSuitabilityModel(parameters=parameters, data=data)
idata_dict = suitability_model.fit_temperature_responses(
    tune=10000, draws=10000, thin=10
)

In [ ]:
plots = suitability_model.plot_fitted_temperature_responses(
    temperature_vals=np.linspace(0, 50, 500)
).cols(2)
plots[0].opts(ylim=(0, 15))
plots[1].opts(ylim=(0, 0.2))
plots[2].opts(ylim=(0, 1))
plots[3].opts(ylim=(0, 50))
plots

In [ ]:
az.plot_trace(idata_dict["eggs_per_female_per_day"]);

In [ ]:
az.plot_trace(idata_dict["development_rate"]);

In [ ]:
az.plot_trace(
    idata_dict["egg_to_adult_survival_probability"],
    var_names=["steepness", "temperature_min", "temperature_max", "noise_std"],
);

In [ ]:
az.plot_trace(idata_dict["egg_to_adult_survival_probability"]);

In [ ]:
az.plot_trace(
    idata_dict["adult_lifespan"],
    var_names=["steepness", "temperature_min", "temperature_max", "noise_std"],
);

In [ ]:
for parameter_name, idata in idata_dict.items():
    print(f"Parameter: {parameter_name}")
    print(az.rhat(idata))

In [ ]:
for parameter_name, idata in idata_dict.items():
    print(f"Parameter: {parameter_name}")
    print(az.ess(idata))